## Initial imports 

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, os, pickle
from sklearn.datasets import load_svmlight_file, dump_svmlight_file
from doptools.chem.solvents import SolventVectorizer
from chython import smiles

# Chemical features

## CircuS and ChyLine descriptors

CircuS (*Circu*lar *S*ubstructures) are fragment descriptors that count circular substructures (atoms and their connected environments) within certain radius. ChyLine (*Chy*thon *Line*ar) are linear fragments, that account for linear substructures of different sizes. The fragment structure are kept as columns in the DatFrame in SMILES format. The following example show how they work.

First, the data set of molecules will be loaded. Here we use a data set of photoswitches from (https://doi.org/10.1039/D2SC04306H). Only data with recorded $\lambda(E_{\pi-\pi*})$ values will be used.

In [50]:
photoswitches = pd.read_table("../examples/photoswitches.csv", sep=",", index_col=0)
data_lambda = photoswitches[pd.notnull(photoswitches['E isomer pi-pi* wavelength in nm'])]
data_lambda

,SMILES,rate of thermal isomerisation from Z-E in s-1,logRate,Solvent used for thermal isomerisation rates,Z PhotoStationaryState,E PhotoStationaryState,E isomer pi-pi* wavelength in nm,Extinction,E isomer n-pi* wavelength in nm,Extinction coefficient in M-1 cm-1,...,TPSSh/6-31G** DFT Z isomer pi-pi* wavelength in nm,TPSSh/6-31G** DFT Z isomer n-pi* wavelength in nm,CAM-B3LYP/6-31G** DFT E isomer pi-pi* wavelength in nm,CAM-B3LYP/6-31G** DFT E isomer n-pi* wavelength in nm,CAM-B3LYP/6-31G** DFT Z isomer pi-pi* wavelength in nm,CAM-B3LYP/6-31G** DFT Z isomer n-pi* wavelength in nm,BHLYP/6-31G* DFT E isomer pi-pi* wavelength in nm,BHLYP/6-31G* DFT E isomer n-pi* wavelength in nm,BHLYP/6-31G* Z isomer pi-pi* wavelength in nm,BHLYP/6-31G* DFT Z isomer n-pi* wavelength in nm
0,C[N]1N=NC(=N1)N=NC2=CC=CC=C2,2.100000e-07,-6.68,MeCN,76.0,72.0,310.0,1.67,442.0,0.0373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C[N]1C=NC(=N1)N=NC2=CC=CC=C2,3.800000e-07,-6.42,MeCN,90.0,84.0,310.0,1.87,438.0,0.0505,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C[N]1C=CC(=N1)N=NC2=CC=CC=C2,1.100000e-07,-6.96,MeCN,98.0,97.0,320.0,1.46,425.0,0.0778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C[N]1C=C(C)C(=N1)N=NC2=CC=CC=C2,1.500000e-06,-5.82,MeCN,96.0,87.0,325.0,1.74,428.0,0.0612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C[N]1C=C(C=N1)N=NC2=CC=CC=C2,7.600000e-09,-8.12,MeCN,98.0,70.0,328.0,1.66,417.0,0.0640,...,295.0,410.0,305.0,427.0,256.0,401.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,OC%38=C%39N=CC=CC%39=C(/N=N/C%40=NC%41=CC(C)=C...,NaN,NaN,NaN,NaN,NaN,456.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401,OC%42=C%43N=CC=CC%43=C(/N=N/C%44=NC%45=CC=CC=C...,NaN,NaN,NaN,NaN,NaN,437.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,N#CC1C(SC(/N=N/C2=NC(C=CC([N+]([O-])=O)=C3)=C3...,NaN,NaN,NaN,NaN,NaN,545.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,N#Cc5c(c6ccc(Cl)cc6)c(/N=N/C7=NC(C=CC([N+]([O-...,NaN,NaN,NaN,NaN,NaN,535.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To transform the SMILES into descriptors, the molecules need to be first transformed into Chython Molecule objects, with the *smiles* function. This also allows us to visualize the molecules.

In [51]:
lambda_mols = [smiles(s) for s in data_lambda.SMILES]
[m.canonicalize() for m in lambda_mols]
[m.clean2d() for m in lambda_mols]
lambda_mols[0]

In [65]:
from doptools import ChythonCircus

circus_fragmentor = ChythonCircus(0, # minimum radius
                                  3) # maximum radius
# using fit function of sklearn Transformer
circus_fragmentor.fit(lambda_mols)
# using transform function of sklearn Transformer
circus_descriptors = circus_fragmentor.transform(lambda_mols) 
circus_descriptors

,C,N,CN,nn(n)C,nnn,n(c)n,nc(n)N,N(=N)C,cc(c)N,ccc,...,s1ccc(C)c1N=Nc(s)n,c1(sccn1)N=Nc(s)c,c1(c(C)c(N)sc1N=N)C,C(#N)c1c(N)sc(N)c1C,C(CCC)(O)C,CCC(CC)OC,CC(C)OC,C1CC(CCC1O)C(C)C,C1CC(CCC1C)OC,C1CC(CCC1)OC
0,8,6,1,2,1,2,1,2,1,5,...,0,0,0,0,0,0,0,0,0,0
1,9,5,1,0,0,1,1,2,1,5,...,0,0,0,0,0,0,0,0,0,0
2,10,4,1,0,0,1,0,2,1,6,...,0,0,0,0,0,0,0,0,0,0
3,11,4,1,0,0,1,0,2,1,5,...,0,0,0,0,0,0,0,0,0,0
4,10,4,1,0,0,1,0,2,2,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,18,4,0,0,0,0,0,2,1,6,...,0,0,0,0,0,0,0,0,0,0
388,16,5,0,0,0,0,1,2,1,8,...,0,0,0,0,0,0,0,0,0,0
389,18,5,1,0,0,0,0,2,0,3,...,0,0,0,0,0,0,0,0,0,0
390,18,5,1,0,0,0,0,2,0,7,...,1,1,1,1,0,0,0,0,0,0


The calculator can also use SMILES directly, although an additional parameter *fmt* needs to be used.

In [53]:
circus_fragmentor_smi = ChythonCircus(0, # minimum radius
                                      3, # maximum radius
                                     fmt="smiles") # indicating that SMILES should be used directly

circus_fragmentor_smi.fit(data_lambda.SMILES) # using fit function of sklearn Transformer
circus_descriptors = circus_fragmentor_smi.transform(data_lambda.SMILES) # using transform function of sklearn Transformer
circus_descriptors

,C,N,CN,NN(N)C,NN=N,N(=N)C,NC(N)=N,N(N)=C,CC(N)=C,C(=C)C,...,c1(c)ccsc1N=NC(=N)S,C=1N=C(N=Nc(s)c)SC=1,c1(c(c(c)c(N=N)s1)C)N,c1(c(c(c(N)s1)C#N)c)N,C(CCC)(O)C,CCC(CC)OC,CC(C)OC,C1CC(CCC1O)C(C)C,C1CC(CCC1C)OC,C1CC(CCC1)OC
0,8,6,1,2,1,3,1,1,1,5,...,0,0,0,0,0,0,0,0,0,0
1,9,5,1,0,0,2,1,1,1,5,...,0,0,0,0,0,0,0,0,0,0
2,10,4,1,0,0,2,0,1,1,6,...,0,0,0,0,0,0,0,0,0,0
3,11,4,1,0,0,2,0,1,1,5,...,0,0,0,0,0,0,0,0,0,0
4,10,4,1,0,0,2,0,1,2,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,18,4,0,0,0,2,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
388,16,5,0,0,0,2,1,0,4,8,...,0,0,0,0,0,0,0,0,0,0
389,18,5,1,0,0,2,0,0,1,3,...,0,0,0,0,0,0,0,0,0,0
390,18,5,1,0,0,2,0,0,1,3,...,1,1,1,1,0,0,0,0,0,0


In [66]:
from doptools import ChythonLinear

chyline_fragmentor = ChythonLinear(2, # minimum length
                                   8) # maximum length
# using fit function of sklearn Transformer
chyline_fragmentor.fit(lambda_mols) 
# using transform function of sklearn Transformer
chyline_descriptors = chyline_fragmentor.transform(lambda_mols) 
chyline_descriptors

,c:c:c:c,c:c:c:c:c,c:c:c:c:cN=Nc,c:c,n:nC,n:cN=Nc:c:c,Nc,Nc:c,n:n:n:cN=N,n:n:cN=Nc:c,...,N#Cc:c:c:s:c,c:n:cN=Nc:cc,N#Cc:c:s:c,N#Cc:c:s:cN=N,N#Cc:c:s:c:c,Cc:cc,Nc:s:cN=Nc,n:cN=Nc:s:cN,OCCCCCCC,NCCCCCCO
0,6,6,2,6,2,4,3,2,2,4,...,0,0,0,0,0,0,0,0,0,0
1,6,6,2,6,1,4,3,2,0,2,...,0,0,0,0,0,0,0,0,0,0
2,6,6,2,8,1,2,3,3,0,2,...,0,0,0,0,0,0,0,0,0,0
3,6,6,2,8,1,2,3,3,0,2,...,0,0,0,0,0,0,0,0,0,0
4,6,6,2,8,1,0,3,4,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,17,18,3,15,0,3,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
388,17,18,3,15,0,6,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
389,6,6,0,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
390,13,12,0,15,0,1,3,2,0,0,...,1,1,1,1,1,1,1,1,0,0


## Fingeprint calculation

Fingerprint calculation in DOPtools is presented by Fingerprinter class, an umbrella class that can calculate several different types of fingerpints. The type itself is defined in *fp_type* parameter. The size of vectors is defined in *nBits*, with 1024 by default. *radius* defined the radius, length or otherwise size of Morgan and RDkit fingerprints and is not used for any other type. Any additional parameters should be given as dictionary into *params* argument. The functionality is otherwise the same and CircuS or ChyLine.

In [55]:
from doptools import Fingerprinter

# Avalon fingperints
avalon_calc = Fingerprinter(fp_type="avalon")
avalon_fp = avalon_calc.fit_transform(lambda_mols)
print("Avalon size:", avalon_fp.shape)

# Torsion fingerprints
torsion_calc = Fingerprinter(fp_type="torsion")
torsion_fp = torsion_calc.fit_transform(lambda_mols)
print("Torsion size:", torsion_fp.shape)

# Morgan fingerprints
morgan_calc_2 = Fingerprinter(fp_type="morgan",
                             radius=2)
morgan_fp_2 = morgan_calc_2.fit_transform(lambda_mols)
print("Morgan, radius 2, size:", morgan_fp_2.shape)

morgan_calc_3 = Fingerprinter(fp_type="morgan",
                             radius=3)
morgan_fp_3 = morgan_calc_3.fit_transform(lambda_mols)
print("Morgan, radius 3, size:", morgan_fp_3.shape)

# Adding Morgan features to fingerprints
morganf_calc_3 = Fingerprinter(fp_type="morgan",
                              radius=3, 
                              params={"useFeatures":True}) # additional parameters should be given in a dictionary here
morganf_fp_3 = morganf_calc_3.fit_transform(lambda_mols)
print("Morgan with features, radius 3, size:", morganf_fp_3.shape)

Avalon size: (392, 1024)
Torsion size: (392, 1024)
Morgan, radius 2, size: (392, 1024)
Morgan, radius 3, size: (392, 1024)
Morgan with features, radius 3, size: (392, 1024)


# Concatenation of features

*ComplexFragmentor* class allows to concatenate several feature types into the same table for one dataset, including the possibility of using different structure columns. Here we show the examples for concatenaition of structural and physico-chemical descriptors. For this, we will use an extract of data from (https://doi.org/10.1002/anie.202218659) related to the catalyst modeling.

In [56]:
cat_data = pd.read_excel("../examples/THP_extract.xls")
cat_data

,name,SMILES,Ar,R,solvent,concentration,T(K),e.r.,ratio(R),ddG,ddG calib (C=0.05)
0,3,FC(F)(F)S(=O)(=O)N=P1(NP2(Oc3c(cc4ccccc4c3-c3c...,[C:2][c:1]1ccccc1,FC(F)(F)S(=O)=O,methylcyclohexane,0.250,353.15,51.6:48.4,0.515861,0.1864,0.2957
1,35,Fc1c(F)c(F)c2c(F)c(c(F)c(F)c2c1F)S(=O)(=O)N=P1...,[C:2][c:1]1ccccc1,Fc1c(F)c(F)c2c(F)c(c(F)c(F)c2c1F)S(=O)=O,cyclohexane,0.250,333.15,25.3:74.7,0.252518,-3.0060,-4.7684
2,5,Cc1ccc(cc1)-c1cc2ccccc2c-2c1OP(NP1(Oc3c(cc4ccc...,[C:2][c:1]1ccc(C)cc1,FC(F)(F)S(=O)=O,cyclohexane,0.250,333.15,47.1:52.9,0.470613,-0.3260,-0.5171
3,36,Cc1ccc(cc1)-c1cc2ccccc2c-2c1OP(NP1(Oc3c(cc4ccc...,[C:2][c:1]1ccc(C)cc1,Fc1c(F)c(F)c(c(F)c1F)S(=O)=O,cyclohexane,0.250,333.15,33.4:66.6,0.333583,-1.9169,-3.0408
4,37,COc1ccc(cc1)-c1cc2ccccc2c-2c1OP(NP1(Oc3c(cc4cc...,COc1cc[c:1]([C:2])cc1,FC(F)(F)S(=O)=O,cyclohexane,0.250,333.15,61.6:38.4,0.616426,1.3141,2.0846
5,8,FC(F)(F)S(=O)(=O)N=P1(NP2(Oc3c(cc4ccccc4c3-c3c...,[C:2][c:1]1ccc2ccccc2c1,FC(F)(F)S(=O)=O,methylcyclohexane,0.250,353.15,41.2:58.8,0.412240,-1.0415,-1.6521
6,38,Cc1cc(C)cc(c1)-c1cc2ccccc2c-2c1OP(NP1(Oc3c(cc4...,[C:2][c:1]1cc(C)cc(C)c1,FC(F)(F)S(=O)=O,methylcyclohexane,0.250,353.15,56.4:43.6,0.563633,0.7514,1.1919
7,6,FC(F)(F)S(=O)(=O)N=P1(NP2(Oc3c(cc4ccccc4c3-c3c...,[C:2][c:1]1cccc2ccccc12,FC(F)(F)S(=O)=O,cyclohexane,0.125,333.15,70.2:29.8,0.702047,2.3740,2.8960
8,39,Fc1c(F)c(F)c(c(F)c1F)S(=O)(=O)N=P1(NP2(Oc3c(cc...,[C:2][c:1]1ccc2ccccc2c1,Fc1c(F)c(F)c(c(F)c1F)S(=O)=O,cyclohexane,0.250,333.15,36.4:63.6,0.364384,-1.5412,-2.4448
9,40,Cc1cc(C)cc(c1)-c1cc2ccccc2c-2c1OP(NP1(Oc3c(cc4...,[C:2][c:1]1cc(C)cc(C)c1,Fc1c(F)c(F)c(c(F)c1F)S(=O)=O,cyclohexane,0.250,333.15,48.1:51.9,0.480504,-0.2161,-0.3428


The catalyst is represented here as a combination of Ar and R substituents. Due to the size differences in these substituents, we want to use different sized CircuS descriptors for them. To do that, we need to add columns with actual molecular structures into the table, as the ComplexFragmentor takes a DataFrame as the input.

In [57]:
ars = [smiles(s) for s in cat_data["Ar"]]
[c.clean2d() for c in ars]

rs = [smiles(s) for s in cat_data["R"]]
[c.clean2d() for c in rs]

cat_data["ar_mol"] = ars
cat_data["r_mol"] = rs

In [58]:
from doptools import ComplexFragmentor

cf = ComplexFragmentor(associator=[
    ("ar_mol", ChythonCircus(0,3)), # the associator connects the column name in the table to the descriptor calculator
    ("r_mol", ChythonCircus(0,1))
])
cf.fit(cat_data) # DataFrame needs to be given as argument, as it will use the associator keys to pick the correct columns
cat_desc = cf.transform(cat_data)
cat_desc

,ar_mol::C,ar_mol::CC,ar_mol::cc(c)C,ar_mol::ccc,ar_mol::c(c)c(cc)C,ar_mol::cc(ccc)C,ar_mol::c(c)ccc,ar_mol::c1ccccc1C,ar_mol::c1ccccc1,ar_mol::c1cc(ccc1C)C,...,r_mol::FC,r_mol::FC(F)(F)S,r_mol::O=S(=O)C,r_mol::S=O,r_mol::cc(c)F,r_mol::cc(c)c,r_mol::cc(c)S,r_mol::FC(F)(F)C,r_mol::CC(C)(F)F,r_mol::FC(F)(S)C
0,7,1,2,5,2,2,3,5,1,0,...,3,1,1,2,0,0,0,0,0,0
1,7,1,2,5,2,2,3,5,1,0,...,7,0,1,2,7,2,1,0,0,0
2,8,2,4,4,4,4,0,2,0,4,...,3,1,1,2,0,0,0,0,0,0
3,8,2,4,4,4,4,0,2,0,4,...,5,0,1,2,5,0,1,0,0,0
4,8,1,2,4,2,2,0,1,0,0,...,3,1,1,2,0,0,0,0,0,0
5,11,1,2,7,2,1,2,0,0,0,...,3,1,1,2,0,0,0,0,0,0
6,9,3,6,3,6,0,0,0,0,0,...,3,1,1,2,0,0,0,0,0,0
7,11,1,2,7,0,1,3,0,0,0,...,3,1,1,2,0,0,0,0,0,0
8,11,1,2,7,2,1,2,0,0,0,...,5,0,1,2,5,0,1,0,0,0
9,9,3,6,3,6,0,0,0,0,0,...,5,0,1,2,5,0,1,0,0,0


The columns of the table contain SMILES of the fragments, preceded by the name of the column where the structure comes from, separated by "::". 

SMILES columns can still be used to calculate descriptors.

In [59]:
cf2 = ComplexFragmentor(associator=[
    ("Ar", ChythonCircus(0,3,fmt="smiles")), # the associator connects the column name in the table to the descriptor calculator
    ("R", ChythonCircus(0,1,fmt="smiles"))
])
cf2.fit(cat_data) # DataFrame needs to be given as argument, as it will use the associator keys to pick the correct columns
cat_desc2 = cf2.transform(cat_data)
cat_desc2

,Ar::C,Ar::CC,Ar::cc(c)C,Ar::ccc,Ar::c(c)c(cc)C,Ar::cc(ccc)C,Ar::c(c)ccc,Ar::c1ccccc1C,Ar::c1ccccc1,Ar::c1cc(ccc1C)C,...,R::FC,R::FC(F)(F)S,R::O=S(=O)C,R::S=O,R::cc(c)F,R::cc(c)c,R::cc(c)S,R::FC(F)(F)C,R::CC(C)(F)F,R::FC(F)(S)C
0,7,1,2,5,2,2,3,5,1,0,...,3,1,1,2,0,0,0,0,0,0
1,7,1,2,5,2,2,3,5,1,0,...,7,0,1,2,7,2,1,0,0,0
2,8,2,4,4,4,4,0,2,0,4,...,3,1,1,2,0,0,0,0,0,0
3,8,2,4,4,4,4,0,2,0,4,...,5,0,1,2,5,0,1,0,0,0
4,8,1,2,4,2,2,0,1,0,0,...,3,1,1,2,0,0,0,0,0,0
5,11,1,2,7,2,1,2,0,0,0,...,3,1,1,2,0,0,0,0,0,0
6,9,3,6,3,6,0,0,0,0,0,...,3,1,1,2,0,0,0,0,0,0
7,11,1,2,7,0,1,3,0,0,0,...,3,1,1,2,0,0,0,0,0,0
8,11,1,2,7,2,1,2,0,0,0,...,5,0,1,2,5,0,1,0,0,0
9,9,3,6,3,6,0,0,0,0,0,...,5,0,1,2,5,0,1,0,0,0


Solvent descriptors can also be concatenated into the table. The solvent descriptors are physico-chemical parameters (acidity, basicity, polarizability, dipolarity). These are present as tabular values and require specific names of solvents in the column.

In [72]:
from doptools import ComplexFragmentor
from doptools import SolventVectorizer

# this calculator doesn't fit anything, so it can be instantiated before the calculation
sv = SolventVectorizer() 
# the associator connects the column name in the table to the descriptor calculator
cf = ComplexFragmentor(associator=[
    ("ar_mol", ChythonCircus(0,3)), 
    ("r_mol", ChythonCircus(0,1)),
    ("solvent",sv) # added the calcualtion of solvent descriptors 
])
# DataFrame needs to be given as argument, 
# as it will use the associator keys to pick the correct columns
cf.fit(cat_data) 
cat_desc = cf.transform(cat_data)
# only some columns are shown for demonstration purposes
cat_desc[['ar_mol::C',  'ar_mol::cc(c)C', 
       'ar_mol::c(c)c(cc)C','r_mol::FC(F)(F)C',
       'r_mol::CC(C)(F)F',  'solvent::SP Katalan',
       'solvent::SA Katalan', 'solvent::SB Katalan']]

,ar_mol::C,ar_mol::cc(c)C,ar_mol::c(c)c(cc)C,r_mol::FC(F)(F)C,r_mol::CC(C)(F)F,solvent::SP Katalan,solvent::SA Katalan,solvent::SB Katalan
0,7,2,2,0,0,0.675,0,0.069
1,7,2,2,0,0,0.683,0,0.073
2,8,4,4,0,0,0.683,0,0.073
3,8,4,4,0,0,0.683,0,0.073
4,8,2,2,0,0,0.683,0,0.073
5,11,2,2,0,0,0.675,0,0.069
6,9,6,6,0,0,0.675,0,0.069
7,11,2,0,0,0,0.683,0,0.073
8,11,2,2,0,0,0.683,0,0.073
9,9,6,6,0,0,0.683,0,0.073


## Calculation of fragments with dynamic bonds

In [74]:
# reaction must be mapped
# reaction SMILES are cut due to size
r_smiles = "[OH:4][CH2:13][CH2:12][CH2:11][CH2:10][C:2](=[CH2:3])[C:1]1=[CH:5][CH:6]=[CH:7][CH:8]=[CH:9]1>>[CH3:3][C:2]1([CH2:10][CH2:11][CH2:12][CH2:13][O:4]1)[C:1]1=[CH:5][CH:6]=[CH:7][CH:8]=[CH:9]1"
reac = smiles(r_smiles)
circus_fragmentor_r = ChythonCircus(0, # minimum radius
                                  3) # maximum radius
# using fit_transform function of sklearn Transformer
circus_fragmentor_r.fit_transform([reac])

,C,O,CC(C)=C,CC(C)([.>-]O)[=>-]C,C[=>-]C,CO[.>-]C,C=CC,CCC,C(O)C,C(C=C)(C([=>-]C)(C)[.>-]O)=CC,...,OCCCC,C(O[.>-]C)CC,C1=CC(C([=>-]C)([.>-]OC)CC)=CC=C1,C[=>-]C1([.>-]OCCCC1)C(C=C)=CC,C[=>-]C1([.>-]OCCCC1)C(=C)C,C1=CC(C(C)([=>-]C)[.>-]O)=CC=C1,C1=CC(=CC=C1)C,C1=CC=CC=C1,C[=>-]C1([.>-]OCCCC1)C,C[.>-]1CCCCO[.>-]1
0,12,1,1,2,1,1,5,3,1,1,...,1,1,1,1,2,2,2,1,2,1


# Command line interface to calculate descriptors

Normally these scripts are supposed to be launched from the terminal and serve as beckend for server calculations. However, an example is given here for illustration purposes. It calculates the Morgan fingeprrints of three different radii for the dataset of photoswitches from before. The script skips all the row that don't have the indicated property value, so the descriptors will only be calucated for the molecules that have them. 

In [63]:
import subprocess

result = subprocess.run(['launch_preparer.py', 
                         '-i', '../examples/photoswitches.csv', # the input file
                         '--structure_col', 'SMILES', # the name of the column that contains the structures
                         '--property_col', 'E isomer pi-pi* wavelength in nm', #the name of the column that contains the property
                         '--property_names', 'Epipi', # the alias for the property name to be used in the file names
                         '--morgan',  # indicate that the Morgan fingerprints will be calculated
                         '--morgan_nBits', '1024', # the length of the Morgan FP vector
                         '--morgan_radius', '2', '3', '4', # the raddi for Morgan FP, several different can be given at the same time
                         '-o', 'lambda/morgan', # the name of the output directory
                         ], stdout=subprocess.PIPE)
print(result.stdout.decode())


The output directory lambda/morgan created
'E isomer pi-pi* wavelength in nm' column warning: only 392 out of 405 instances have the property.
Molecules that don't have the property will be discarded from the set.



Full list of parameters can be seen using the help function (*-h*) of the script.

In [62]:
result = subprocess.run(['launch_preparer.py', '-h'], stdout=subprocess.PIPE)
print(result.stdout.decode())

usage: Descriptor calculator [-h] -i INPUT [--structure_col STRUCTURE_COL]
                             [--concatenate CONCATENATE [CONCATENATE ...]]
                             --property_col PROPERTY_COL [PROPERTY_COL ...]
                             [--property_names PROPERTY_NAMES [PROPERTY_NAMES ...]]
                             [--standardize] -o OUTPUT [-f {svm,csv}]
                             [-p PARALLEL] [-s] [--separate_folders]
                             [--load_config LOAD_CONFIG] [--morgan]
                             [--morgan_nBits MORGAN_NBITS [MORGAN_NBITS ...]]
                             [--morgan_radius MORGAN_RADIUS [MORGAN_RADIUS ...]]
                             [--morganfeatures]
                             [--morganfeatures_nBits MORGANFEATURES_NBITS [MORGANFEATURES_NBITS ...]]
                             [--morganfeatures_radius MORGANFEATURES_RADIUS [MORGANFEATURES_RADIUS ...]]
                             [--rdkfp]
                             [

Example of calculation of CircuS fragments with saving the pickled objects for fragmentors.

In [64]:
result = subprocess.run(['launch_preparer.py', 
                         '-i', '../examples/photoswitches.csv', # the input file
                         '--structure_col', 'SMILES', # the name of the column that contains the structures
                         '--property_col', 'E isomer pi-pi* wavelength in nm', #the name of the column that contains the property
                         '--property_names', 'Epipi', # the alias for the property name to be used in the file names
                         '--circus',  
                         '--circus_min', '0', 
                         '--circus_max', '2', '3', '4', '5', 
                         '-o', 'lambda/circus', # the name of the output directory
                         '--save' # indicates that the pickled objects of fragmentors housld be saved
                         ], stdout=subprocess.PIPE)
print(result.stdout.decode())

The output directory lambda/circus created
'E isomer pi-pi* wavelength in nm' column warning: only 392 out of 405 instances have the property.
Molecules that don't have the property will be discarded from the set.

